# 7.1. Разнообразие

Как известно, для формирования случайного леса необходимо разнообразие каждого из решающих деревьев в его составе. Добиться такого разнообразия можно как при помощи варьирования параметров деревьев, так и при помощи варьирования обучающих выборок. Это обычно делается при помощи комбинирования методов случайных подпространств и беггинга. Метод случайных подпространств состоит в случайном выборе подмножества признаков. Бэггинг - это техника отбора подмножества объектов из исходной выборки, которая состоит в последовательном случайном выборе объектов выборки [**с возвращением**](https://ru.wikipedia.org/wiki/Размещение#Размещение_с_повторениями).

Для ясности приведём следующий пример:

Пусть  наша выборка - это черный мешок с пронумерованными шарами. Каждый шар символизирует некоторый объект нашей выборки с соответствующим номером. Процедура бэггинга предлагает нам последовательно хорошо перемешав шары в мешке вытаскивать их один за другим не глядя, записывать их номера на лист бумаги, а затем возвращать их назад в мешок, повторяя эту операцию столько раз, сколько объектов содержится в нашей выборке. Затем мы вынем из мешка все те шары, номер которых хотя бы раз возник в нашем списке. Понятно, что скорее всего среди этих номеров будут повторяющиеся, то есть какие-то шары мы вытянем несколько раз, а какие-то - вообще ни разу. Именно поэтому наша подвыборка и получится случайной. Статистически обоснована оценка того, какой процент шаров из исходной выборки в среднем попадёт в итоговую подвыборку. Эта оценка приблизительно равна $62\%$.

Ваша задача - написать реализацию трёх функций и объединить их в класс `sample`, возвращающий по выборке некоторую случайную подвыборку, пригодную для обучения  одного из деревьев в ансамбле случайного леса.

**Замечание:** обратите внимание, что объекты в итоговой подвыборке не должны дублироваться. Мы предлагаем Вам ознакомиться с функциями [np.random.choice](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) и [np.unique](https://numpy.org/doc/stable/reference/generated/numpy.unique.html), они могут оказаться полезны при выполнении этого задания.

Подробнее об этом методе можно прочитать в нашей [лекции](https://colab.research.google.com/drive/1LrqEyfmOKJQdvgxZ56qPcJ_YjJFnP_Ka#scrollTo=kYjii_stqfJo)


In [ ]:
import numpy as np

class sample(object):
    def __init__(self, X, n_subspace):
        self.idx_subspace = self.random_subspace(X, n_subspace)

    def __call__(self, X, y):
        idx_obj = self.bootstrap_sample(X)
        X_sampled, y_sampled = self.get_subsample(X, y, self.idx_subspace, idx_obj)
        return X_sampled, y_sampled

    @staticmethod
    def bootstrap_sample(X):
        """
        Возвращает массив индексов объектов, выбранных с помощью бэггинга (с возвращением)
        """
        n_samples = X.shape[0]
        idx = np.random.choice(np.arange(n_samples), size=n_samples, replace=True)
        return np.unique(idx) 

    @staticmethod
    def random_subspace(X, n_subspace):
        """
        Возвращает массив индексов случайно выбранных признаков (без повторений)
        """
        n_features = X.shape[1]
        return np.random.choice(np.arange(n_features), size=n_subspace, replace=False)

    @staticmethod
    def get_subsample(X, y, idx_subspace, idx_obj):
        """
        Возвращает подвыборку по выбранным индексам объектов и признаков
        """
        X_sampled = X[np.ix_(idx_obj, idx_subspace)]
        y_sampled = y[idx_obj]
        return X_sampled, y_sampled

# Пример формата входных и выходных данных

In [10]:
X = np.array([[1,2,3], [4,5,6], [7,8,9]])
Y = np.array([1, 2, 3])
s = sample(X, 2)

bootstrap_indices = s.bootstrap_sample(X)
X_sampled, y_sampled = s.get_subsample(X, Y, s.idx_subspace, bootstrap_indices)

In [11]:
# Строки, выбранные из исходного массива X
bootstrap_indices

array([0, 1, 2])

In [12]:
# Столбцы, выбранные из исходного массива X
s.idx_subspace

array([2, 1])

In [13]:
# Матрица на выходе
X_sampled

array([[3, 2],
       [6, 5],
       [9, 8]])

In [14]:
# Вектор ответов на выходе
y_sampled

array([1, 2, 3])

## Примечания

1. Не забывайте, что в качестве ответа на задание нужно отправлять именно реализованный класс, соответствующиий [шаблону выше](https://colab.research.google.com/drive/1LoXr0Rwmcivgla2gAzOKuLgh2ISlokvv#scrollTo=juW_Uoo_ic0z&line=5&uniqifier=1).

2. Подумайте, что должны возвращать методы bootstrap_sample и random_subspace (одна из функций возвращает индексы строк, другая - индексы столбцов).

3. Пример из блокнота не обязан совпадать с входными данными из блокнота. Он дан лишь для проверки алгоритмов в указанном формате.

4. Подумайте, в каких реализуемых методах нужен numpy.unique(), а в каких - нет.

5. В реализуемых методах запрещается использовать вывод любой информации на экран (в частности, недопустимо использование print()).

# 7.2. Случайный лес

Ваша задача - написать класс `random_forest` для решения задачи классификации на основе датасета Ирисов Фишера (`sklearn.datasets.load_iris`), принимающий на вход конструктора аргументы `n_estimators`, `max_depth`, `subspaces_dim` и `random_state`. описание этих аргументов приведено ниже. У этого класса должны быть определены методы `.fit()` и `.predict()`, а также поле `._estimators`, в котором должен храниться список алгоритмов, используемых в ансамбле.

- n_estimators - число деревьев в ансамбле
- max_depth - максимальная глубина каждого дерева в ансамбле
- subspaces_dim - размерность случайного подпространства для каждого дерева

Для создания обучающей подвыборки для каждого из базовых классификаторов, Вы можете воспользоваться классом `sample`, который Вы реализовали в прошлом задании. В случае его использования, не забудьте включить его описание в файл с Вашим решением текущего задания. Мы также предлагаем вам организовать выбор подпространств для каждого дерева посредством заполнения списка `subspace_idx`, в котором будут логироваться выбранные для каждого базового классификатора подпространства.

Замечание: в рамках выполнения данного задания запрещено использовать класс `sklearn.ensemble.RandomForestClassifier`. Такой код не пройдёт проверку.

Подберите также гиперпараметры, на которых ваш алгоритм получает наилучшее качество (с точки зрения метрики accuracy, доли правильных ответов) на тестовой выборке с параметром `test_size`=0.3, задайте их в виде глобальных переменных N_ESTIMATORS, MAX_DEPTH, SUBSPACE_DIM.

Шаблон класса:

In [15]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

np.random.seed(42)

# Best hyperparameters found through experimentation
N_ESTIMATORS = 100
MAX_DEPTH = 5
SUBSPACE_DIM = 2

class sample(object):
    def __init__(self, X, n_subspace):
        self.idx_subspace = self.random_subspace(X, n_subspace)

    def __call__(self, X, y):
        idx_obj = self.bootstrap_sample(X)
        X_sampled, y_sampled = self.get_subsample(X, y, self.idx_subspace, idx_obj)
        return X_sampled, y_sampled

    @staticmethod
    def bootstrap_sample(X):
        n_samples = X.shape[0]
        idx = np.random.choice(np.arange(n_samples), size=n_samples, replace=True)
        return np.unique(idx)

    @staticmethod
    def random_subspace(X, n_subspace):
        n_features = X.shape[1]
        return np.random.choice(np.arange(n_features), size=n_subspace, replace=False)

    @staticmethod
    def get_subsample(X, y, idx_subspace, idx_obj):
        X_sampled = X[np.ix_(idx_obj, idx_subspace)]
        y_sampled = y[idx_obj]
        return X_sampled, y_sampled

class random_forest(object):
    def __init__(self, n_estimators: int, max_depth: int, subspaces_dim: int, random_state: int):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.subspaces_dim = subspaces_dim
        self.random_state = random_state
        self._estimators = []
        self.subspace_idx = []
        np.random.seed(random_state)

    def fit(self, X, y):
        self._estimators = []
        self.subspace_idx = []
        
        for _ in range(self.n_estimators):
            # Create bootstrap sample with random subspace
            sampler = sample(X, self.subspaces_dim)
            X_sample, y_sample = sampler(X, y)
            self.subspace_idx.append(sampler.idx_subspace)
            
            # Train decision tree
            tree = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state)
            tree.fit(X_sample, y_sample)
            self._estimators.append(tree)

    def predict(self, X):
        predictions = np.zeros((X.shape[0], len(self._estimators)))
        
        for i, tree in enumerate(self._estimators):
            # Predict using only the features selected for this tree
            X_subspace = X[:, self.subspace_idx[i]]
            predictions[:, i] = tree.predict(X_subspace)
        
        # Majority vote
        final_predictions = np.apply_along_axis(
            lambda x: np.bincount(x.astype(int)).argmax(),
            axis=1,
            arr=predictions
        )
        return final_predictions

# # Example usage
# if __name__ == "__main__":
#     iris = load_iris()
#     X, y = iris.data, iris.target
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
#     rf = random_forest(
#         n_estimators=N_ESTIMATORS,
#         max_depth=MAX_DEPTH,
#         subspaces_dim=SUBSPACE_DIM,
#         random_state=42
#     )
#     rf.fit(X_train, y_train)
#     y_pred = rf.predict(X_test)
    
#     print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

## Примечания

1. В данной задаче запрещено использовать библиотеку pandas.

2. В реализуемых методах запрещается использовать вывод любой информации на экран (в частности, недопустимо использование print()).

# Пример входных и выходных данных

In [16]:
X

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [17]:
X.shape

(3, 3)

In [18]:
y.shape

(150,)

In [19]:
rf = random_forest(25, 15, 2, 42)
rf.fit(X, y)

In [20]:
preds = rf.predict(X)

In [21]:
preds

array([0, 0, 0])